In [1]:
import numpy as np

In [2]:
npzdata=np.load("data.npz")

In [3]:
npzdata

In [4]:
data=npzdata['data']
labels=npzdata['labels']

In [5]:
data

array([[ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       ..., 
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int16)

In [6]:
labels

array([u'9gag.com', u'9gag.com', u'9gag.com', ..., None, None, None], dtype=object)

In [7]:
data.shape

(249996, 5000)

In [8]:
import keras.preprocessing.sequence as sq

Using TensorFlow backend.


In [9]:
maxlen=150

In [10]:
data=sq.pad_sequences(data,maxlen=maxlen,padding='post',truncating='post',dtype='float64')

In [11]:
data=data.reshape(data.shape[0],data.shape[1],1)

In [12]:
from keras.utils import np_utils

In [13]:
dict_label={}

In [14]:
n=0

In [15]:
set_labels=list(set(labels))

In [16]:
for l in set_labels:
    dict_label[l]=n
    n+=1

In [17]:
dict_label

{None: 41,
 u'9gag.com': 77,
 u'abs-cbn.com': 99,
 u'adf.ly': 70,
 u'adobe.com': 39,
 u'aliexpress.com': 22,
 u'allegro.pl': 83,
 u'amazon.com': 50,
 u'amazonaws.com': 62,
 u'aol.com': 72,
 u'apple.com': 69,
 u'archive.org': 9,
 u'askcom.me': 71,
 u'battle.net': 96,
 u'blastingnews.com': 7,
 u'booking.com': 57,
 u'breitbart.com': 76,
 u'bukalapak.com': 21,
 u'businessinsider.com': 48,
 u'conservativetribune.com': 56,
 u'dailymail.co.uk': 61,
 u'dailymotion.com': 29,
 u'detik.com': 59,
 u'deviantart.com': 82,
 u'dictionary.com': 27,
 u'digikala.com': 35,
 u'doubleclick.net': 25,
 u'doublepimp.com': 89,
 u'ebay.com': 49,
 u'espncricinfo.com': 5,
 u'exoclick.com': 55,
 u'extratorrent.cc': 64,
 u'facebook.com': 88,
 u'feedly.com': 32,
 u'gamepedia.com': 95,
 u'github.com': 15,
 u'go.com': 94,
 u'godaddy.com': 93,
 u'goodreads.com': 81,
 u'google.com': 98,
 u'hclips.com': 28,
 u'hola.com': 24,
 u'hotmovs.com': 78,
 u'imdb.com': 97,
 u'instructure.com': 80,
 u'intuit.com': 37,
 u'kompas.com'

In [18]:
labels_new=[]

In [19]:
for l in labels:
    labels_new.append(dict_label[l])

In [20]:
label_1=np_utils.to_categorical(labels_new)

In [21]:
label_1

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [22]:
def generateData(data,labels,batch_size=32):
    nb_instances = data.shape[0]
    nb_classes = labels.shape[1]
    sample_shape = data[0].shape
    batch_data_shape = tuple([batch_size] + list(sample_shape))
    batch_label_shape = (batch_size, nb_classes)
    # Infinite loop
    while True:
        # Generate an exploration order
        indices = np.arange(nb_instances)

        np.random.shuffle(indices)

        # Generate batches
        imax = int(len(indices) / batch_size)
        for i in range(imax):
            # Form a batch
            x = np.empty(batch_data_shape)
            y = np.empty(batch_label_shape)
            for j, k in enumerate(indices[i * batch_size: (i + 1) * batch_size]):
                x[j] = data[k]
                y[j] = labels[k]
            if x.shape != batch_data_shape:
                print(x.shape)
                exit(0)
            yield x, y

In [23]:
temp=generateData(data,label_1)

In [35]:
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Input
from keras.models import Sequential
from keras.optimizers import RMSprop

In [46]:
model=Sequential()

In [47]:
input_length=maxlen
input_dim=1

In [48]:
model.add(LSTM(input_shape=(input_length,input_dim),units=128,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=True,dropout=0.2))

In [49]:
model.add(LSTM(units=128,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=False,dropout=0.2))

In [50]:
nb_classes=label_1.shape[1]

In [51]:
nb_classes

101

In [52]:
model.add(Dense(units=nb_classes,activation='softmax'))

In [53]:
metrics=['accuracy']

In [54]:
optimizer=RMSprop(lr=0.001,decay=0.0)

In [55]:
model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=metrics)

In [56]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 150, 128)          66560     
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 101)               13029     
Total params: 211,173
Trainable params: 211,173
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
from sklearn import model_selection

In [58]:
X_train, X_test, y_train, y_test =model_selection.train_test_split(data, label_1, test_size=0.33)

In [59]:
trainGen=generateData(X_train,y_train)

In [60]:
valGen=generateData(X_test,y_test)

In [ ]:
history=model.fit_generator(generator=trainGen,steps_per_epoch=X_train.shape[0]//32,validation_data=valGen,validation_steps=X_test.shape[0]//32,epochs=5)

In [63]:
import matplotlib.pyplot as plt

In [64]:
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label="Training", color='purple')
if val_loss is not None:
    plt.ylim(0, 5)
    plt.plot(history.history['val_loss'], label="Validation", color='green')
plt.title("Loss plot", y=1.08)
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode='expand', borderaxespad=0.)
plt.yticks(np.arange(0, 5, 0.5))
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid()
plt.show()

NameError: name 'history' is not defined

In [65]:
plt.ylim(0, 1)
plt.plot(history.history['acc'], label="Training", color='red')
if val_acc is not None:
    plt.plot(val_acc, label="Validation", color='blue')
plt.title("accuracy plot", y=1.08)
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode='expand', borderaxespad=0.)
plt.yticks(np.arange(0, 1, 0.05))
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid()
plt.show()

NameError: name 'history' is not defined